In [1]:
"""Divide a binarized image into individual connected components."""
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

from __future__ import division
import cv2
import numpy as np
import math
import copy

MIN_HEIGHT = 32
MIN_ASPECT_RATIO = 0.6
MAX_ASPECT_RATIO = 6
TOL = 4


In [2]:
def is_good_roi(roi):
    """Checks the aspect ratio and minimum size of the ROI.
    Returns True if it's OK."""
    x,y,width,height = roi
    return height >= MIN_HEIGHT and MIN_ASPECT_RATIO <= height/width <= MAX_ASPECT_RATIO


In [3]:

def is_overlap(r1, r2):
    """Returns True if the two ROIs overlap."""
    #
    # r1 is always on the left
    #
    if r2[0] < r1[0]:
        r1, r2 = r2, r1
    x1, y1, w1, h1 = r1
    x2, y2, w2, h2 = r2

    if not x1 <= x2 <= x1+w1:
        return False

    return (y1 <= y2 <= y1+h1) or (y2 <= y1 <= y2+h2)

In [4]:
def merge(r1, r2):
    """Merge two ROIs, assuming they overlap."""
    x1, y1, w1, h1 = r1
    x2, y2, w2, h2 = r2

    xx1 = min(x1, x2)
    yy1 = min(y1, y2)
    xx2 = max(x1+w1, x2+w2)
    yy2 = max(y1+h1, y2+h2)

    ww = xx2 - xx1
    hh = yy2 - yy1
    return (xx1, yy1, ww, hh)


In [5]:

def merge_overlaps(rois):
    """Perform a single iteration of overlapping ROIs.
    Returns the number of overlaps merged."""
    merged = []
    used = [False]*len(rois)
    
    for i,r1 in enumerate(rois):
        #print('i,r2')
        if used[i]:
            continue
        for j,r2 in enumerate(rois):
            #print('j,r2')
            if j <= i or used[j]:
                continue
            if is_overlap(r1, r2):
                #print('is_overlap')
                merged.append(merge(r1, r2))
                used[i] = used[j] = True
    num_merged = len(merged)
    #num_merged = len(list(merged))
    for i,r in enumerate(rois):
        if used[i]:
            continue
        merged.append(r)

    return merged, num_merged


In [6]:

def roi_compare(roi1, roi2):
    x1, y1, _, _ = roi1
    x2, y2, _, _ = roi2
    x1 //= 8
    x2 //= 8
    y1 //= 8
    y2 //= 8
    if y1 < y2:
        return -1
    elif y1 > y2:
        return 1

    if x1 < x2:
        return -1
    elif x1 > x2:
        return 1

    return 0

In [7]:

def get_rois(binarized, numiter=1):
    #
    # findContours uses the input image as scratch space
    #
    tmp = copy.deepcopy(binarized)
    #
    # Open the image to get rid of "holes" in the border
    #
    tmp = cv2.erode(cv2.dilate(tmp, None, iterations=numiter), None, iterations=numiter)
    
    
    #contours, _= cv2.findContours(tmp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #_, contours, _ = cv2.findContours(tmp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    #ret,thresh = cv2.threshold(imgray,127,255,0)
    #contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    ret,thresh = cv2.threshold(binarized,127,255,0)
    image,contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #print('counturs: %d' % len(contours))
    #cnt = contours[0]
    #print('countur: %s' % cnt)
    #print(cnt.shape)
    #print(cv2.boundingRect(cnt))
    
    rois = map(lambda x: cv2.boundingRect(x), contours)
    rois = list(rois)
    
    #print (rois)
    #print (len(rois))    
    #rois, num_merged = merge_overlaps(rois)           
    #print(rois)
    #print('done merging')
    #print(num_merged)
    
    print('starting merge')        
    while True:
        rois, num_merged = merge_overlaps(rois)                
        if num_merged == 0:
            break
     
    print('done merging')        
        
    roisf = filter(is_good_roi, rois)
    rois = list(roisf)
    rois.sort(key=roi_compare)  
    
    print('printing final  rois')
    print ( rois)
    
    return rois
    

In [8]:
#im = cv2.imread('document-scanner/images/braums-4.jpg', cv2.IMREAD_GRAYSCALE)
#im = cv2.imread('document-scanner/images/sams.jpg', cv2.IMREAD_GRAYSCALE)
im = cv2.imread('document-scanner/images/ocr-binarized.jpeg', cv2.IMREAD_GRAYSCALE)
rois = get_rois(im)
#color_im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
#print("before roi loop")
#for (x,y,width,height) in rois:
#    print("in roi loop")
#    cv2.rectangle(color_im, (x,y), (x+width, y+height), (255,0,0,0), 2)
#plt.imshow(color_im)
#plt.show()

starting merge
done merging


TypeError: roi_compare() missing 1 required positional argument: 'roi2'

In [ ]:
im = Image.fromarray(color_im)
im.save("document-scanner/images/ocr-cc.jpeg")
